In [23]:
import cv2
import os
import time

image_dir = 'C:\\Users\\front\\Downloads\\imageAI_data' # ローカルの画像データファイルパス


In [24]:
processed_images = []
labels = []
haarcascade_path = r'data\haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haarcascade_path)
print("Using Haar Cascade path:", haarcascade_path)
if not os.path.exists(haarcascade_path):
    raise FileNotFoundError(f'Haar Cascade XML file not found at {haarcascade_path}')
face_cascade = cv2.CascadeClassifier(haarcascade_path)

Using Haar Cascade path: data\haarcascade_frontalface_default.xml


In [25]:
for label, person_name in enumerate(os.listdir(image_dir)):
    person_dir = os.path.join(image_dir, person_name)
    if not os.path.isdir(person_dir):
        continue

    for image_name in os.listdir(person_dir):
        image_path = os.path.join(person_dir, image_name)
        image = cv2.imread(image_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            face = gray_image[y:y+h, x:x+w]
            resized_face = cv2.resize(face, (128, 128))
            processed_images.append(resized_face)
            labels.append(label)


In [26]:
import tensorflow as tf
from keras import layers
from keras import models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(set(labels)), activation='softmax')  # Number of unique labels
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

import numpy as np
processed_images = np.array(processed_images).reshape(-1, 128, 128, 1)
labels = np.array(labels)

model.fit(processed_images, labels, epochs=10, validation_split=0.2)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 209ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 

In [27]:
def recognize_face(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = gray_image[y:y+h, x:x+w]
        resized_face = cv2.resize(face, (128, 128)).reshape(1, 128, 128, 1)
        predictions = model.predict(resized_face)
        label = np.argmax(predictions)
        confidence = np.max(predictions)

        return label, confidence

    return None, None

In [35]:
video_capture = cv2.VideoCapture(0)
Absence = None
name = None
while True:
    ret, frame = video_capture.read()
    label, confidence = recognize_face(frame)

    if label == 0:
        name = "Yudai"
    if confidence == None:
        if Absence == None:
            Absence = time.time()
        elif time.time() - Absence >= 5:
            break
    elif confidence > 0.9:  # しきい値
        Absence = None
        print(f"検出した人物名: {name}")
        print(f"一致率: {confidence}")

video_capture.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
検出した人物名: Yudai
一致率: 1.0
1/1 ━━━━